<a href="https://colab.research.google.com/github/akansh12/Deep-Learning-for-imaging-EE5179/blob/main/Assignment_1/PA1_CS22Z003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST Classification using MLP


Tasks to to:
- Downloading the Dataset
- Baseline: IP-h(500)-h(250)-h(100)-OP
- Activation functions for hidden layers: Sigmoid and for the Output layer it is softmax
- Gradient-Descent
- lr = 0.01, batch size = 64, epoch = 15
-  Extra marks for experimentations
- Glorot Initialization
- Plot the loss for 200 iterations
- Confustion matrix and classification report
1.1
- Use Tanh and RELU

Task 2:
- Use pytorch for the same
- Add L2 Regularization to it.